In [1]:
import pandas as pd
import geopandas as gpd
import networkx as nx
from pathlib import Path
from sloyka import Geocoder,TextClassifiers, City_services, EmotionRecognizer

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thebe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2024-06-03 16:37:22,789 SequenceTagger predicts: Dictionary with 7 tags: O, S-Service, B-Service, E-Service, I-Service, <START>, <STOP>


In [15]:
import warnings

warnings.simplefilter("ignore")

import os
import sys

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

from sloyka.src.data_getter import VKParser

In [174]:
domain = 'newstucentr'
token = '6fddc0e06fddc0e06fddc0e01e6cc5424166fdd6fddc0e0098175a5fbe443fba5b3ffe8'

In [175]:
df = VKParser().run_parser(domain, token, step=100, cutoff_date='2024-03-20')

2024-05-18 22:07:54
2024-05-05 12:30:00
2024-04-18 15:49:37
2024-04-01 16:26:36
2024-03-17 16:18:21
posts downloaded


100%|██████████| 410/410 [05:27<00:00,  1.25it/s]


comments downloaded


c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


In [166]:
df['date'].iloc[0]

Timestamp('2024-06-03 10:08:16')

In [ ]:
df

In [24]:
df.to_csv('..\\sloyka\\sample_data\\test_activity_data.csv', sep=';', encoding='utf-8', ignore_index=True)

In [28]:
df_ = pd.read_csv('..\\sloyka\\sample_data\\test_activity_data.csv', sep=';', index_col=0, encoding='utf-8')

c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\pandas\core\algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])


In [29]:
df_.dropna(subset=['text'], inplace=True)

In [31]:
df_['text'].iloc[835]

'[id836328|Марина], нет , ноя думаю УК уже пора , за неоднократное нарушение штраф по моему до 200.000тр . Самое противное , что ни дворники , ни домоуправ не говорят по чьему распоряжению к нам вывозят баки .\r\nДома 25 А , 27, 28, 29,31,33 по улице Черняховского не везут мусор не под окна своих домов , а к нам ,совершенно на другую улицу , при этом им ни чего не мешает так же выкатывать баки к себе , но там же элита , а в наших домах чернь видимо живет . Да мусор в принципе не должен находится вне контейнерной площадки . ВИдимо УК или ТСЖ дома 25А кому то платит за это , а все остальные этот беспредел покрывают . ТАк тщательно все молчат .\r\nЕсть же территория , которая принадлежит тому или иному дому , которая и должна обслуживаться дворниками этого двора'

In [39]:
from tqdm.auto import tqdm
for i in tqdm(range(len(df_))):
    base_text = df_['text'].iloc[i]
    while '\n' in base_text or '\r' in base_text:
        if '\n' in df_['text'].iloc[i]:
            text = df_['text'].iloc[i]
            base_text = text.replace('\n', ' ')
            df_.at[i, 'text'] = base_text
        if '\r' in df_['text'].iloc[i]:
            text = df_['text'].iloc[i]
            base_text = base_text.replace('\r', ' ')
            df_.at[i, 'text'] = base_text

  0%|          | 0/1074 [00:00<?, ?it/s]

In [44]:
df_.dropna(subset=['text'], inplace=True)

In [47]:
from tqdm.auto import tqdm

ids = []
for i in tqdm(range(len(df_))):
    
    if '\n' in df_['text'].iloc[i] or '\r' in df_['text'].iloc[i]:
        ids.append(df_['id'].iloc[i])

  0%|          | 0/1074 [00:00<?, ?it/s]

In [50]:
normoalized_df = df_.loc[df_['id'].isin(ids)].copy()

c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\pandas\core\algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])


In [51]:
normoalized_df.reset_index(drop=True, inplace=True)

In [ ]:
normoalized_df

In [24]:
g = Geocoder()

2024-06-03 17:09:31,670 SequenceTagger predicts: Dictionary with 5 tags: O, S-ADDRESS, B-ADDRESS, E-ADDRESS, I-ADDRESS


In [52]:
gdf = g.run(osm_id = 338635,
            tags = {"admin_level": ["6"]},
            date = "2024-04-22T00:00:00Z", 
            df=normoalized_df,
            text_column='text',
            group_column='group_name')

06-03 17:43 | INFO     | run_OSM_dfs started


C:\Projects\IDU\sloyka\sloyka\src\geocoder.py:310: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  green_obj = ox.geometries_from_place(osm_city_name, tags)
c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types

06-03 17:43 | INFO     | find_other_geo_obj started


c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\pandas\core\algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])
c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  

06-03 17:46 | INFO     | clear_names started
06-03 17:46 | INFO     | get_street started
06-03 17:46 | INFO     | extract_ner_street started


...


  0%|          | 0/16 [00:00<?, ?it/s]c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
100%|██████████| 16/16 [00:00<00:00, 64.53it/s]

06-03 17:46 | INFO     | get_stem started



c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


06-03 17:47 | INFO     | create_gdf started


100%|██████████| 5/5 [00:02<00:00,  2.16it/s]
C:\Projects\IDU\sloyka\sloyka\src\geocoder.py:1152: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  gdf = pd.merge(gdf, df_obj, how='outer')
c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


In [54]:
gdf

,date,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,...,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry,other_geo_obj,geo_obj_tag
0,2024-04-15 13:20:00,57882.0,"[id16164672|Марина Баскова], здравствуйте! В н...",NaN,0.0,NaN,reply,NaN,-143265175.0,57833.0,...,Суворовский,проспект,Суворовский проспект д60 Санкт-Петербург Россия,['Суворовский проспект д60 Санкт-Петербург Рос...,Суворовский проспект д60 Санкт-Петербург Россия,Суворовский проспект,"Суворовский проспект, Пески, округ Смольнинско...",POINT (30.38186 59.94202),NaN,street
1,2024-03-28 17:34:36,57587.0,"УЖЕ !!!! Сначала 15 марта. бАмажкой от работы,...",NaN,1.0,NaN,comment,NaN,96685058.0,57582.0,...,Литейном,проспект,Литейный проспект Санкт-Петербург Россия,['Литейный проспект Санкт-Петербург Россия'],Литейный проспект Санкт-Петербург Россия,Литейный проспект,"Литейный проспект, Литейный округ, Санкт-Петер...",POINT (30.34812 59.93828),NaN,street
2,2024-04-25 08:01:15,58081.0,"[id1759494|Julia Zhugun], здравствуйте! \r\n\r...",NaN,0.0,NaN,reply,NaN,-143265175.0,57582.0,...,Невский,проспект,Невский проспект д59 Санкт-Петербург Россия,['Невский проспект д59 Санкт-Петербург Россия'],Невский проспект д59 Санкт-Петербург Россия,Невский проспект,"Невский проспект, Пески, округ Смольнинское, С...",POINT (30.36618 59.93053),NaN,street
3,2024-03-27 08:11:52,57555.0,"[id7332491|Татьяна Николаева], здравствуйте!\r...",NaN,1.0,NaN,reply,NaN,-143265175.0,57537.0,...,Таврической,улица,Таврическая улица Санкт-Петербург Россия,['Таврическая улица Санкт-Петербург Россия'],Таврическая улица Санкт-Петербург Россия,Таврическая улица,"Таврическая улица, Пески, округ Смольнинское, ...",POINT (30.37827 59.94871),NaN,street
4,2024-03-29 11:49:58,57605.0,"[id6179030|Ольга Ефимова], здравствуйте! С пон...",NaN,1.0,NaN,reply,NaN,-189318064.0,57537.0,...,Суворовский,проспект,Суворовский проспект Санкт-Петербург Россия,['Суворовский проспект Санкт-Петербург Россия'],Суворовский проспект Санкт-Петербург Россия,Суворовский проспект,"Суворовский проспект, Пески, округ Смольнинско...",POINT (30.38186 59.94202),NaN,street
5,2024-03-18 07:47:56,57383.0,"[id7038519|Светлана Белованова], здравствуйте!...",NaN,0.0,NaN,reply,NaN,-143265175.0,57317.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (30.36747 59.93735),Некрасовский сад,park
6,2024-03-18 07:47:56,57383.0,"[id7038519|Светлана Белованова], здравствуйте!...",NaN,0.0,NaN,reply,NaN,-143265175.0,57317.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (30.33962 59.96925),Гренадерский сад,park


In [7]:
gdf.drop('date', axis=1, inplace=True)

In [25]:
gdf.to_file('..\\sloyka\\sample_data\\test_activity_data.geojson')

In [10]:
gdf['text'].iloc[0]

'Дорого и плохо, а порой еще и опасно  Сюжет «78» о некачественном и дорогом благоустройстве в Петербурге. В этот раз журналисты решили уделить особое внимание нашей местной компании ООО ПСФ «Корт» (и аффилированному с ней ООО «Зеленстрой»).  Её учредителем по документам является бывшая жена Виктора Шабнова, экс-зампредседателя Комитета по благоустройству, уволенного из Смольного в 2018 г. по статье «утрата доверия» за нарушение закона о противодействии коррупции.  Несмотря на такую опалу, семейный бизнес Шабновых вовсю процветает - в последние годы они, без какой-либо конкуренции и снижения цены, забирают все самые жирные контракты по благоустройству в южных районах Петербурга.  Парк Авиаторов (почти 1 млрд рублей), сквер Раисы Штрейс, скейт-парк и [https://vk.com/wall-33468323_41130|«собачий аквапарк»] на Железнодорожной в Пушкине за 78 млн (о котором также упомянуто в этом сюжете), I этап благоустройства «большого круга» в Славянке за 95 млн - список можно бесконечно продолжать. Тол

In [26]:
text_classifier = TextClassifiers('Sandrro/text_to_subfunction_v10')

In [27]:
gdf[['cats', 'probs']] =  gdf["text"].progress_map(lambda x: text_classifier.run_text_classifier(x)).to_list()

  0%|          | 0/387 [00:00<?, ?it/s]c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  1%|          | 2/387 [00:03<11:20,  1.77s/it]c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  1%|          | 3/387 [00:05<10:31,  1.64s/it]c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated an

In [28]:
gdf

,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,parents_stack,...,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry,other_geo_obj,geo_obj_tag,cats,probs
0,58907,В Центральном районе продолжается борьба с нез...,892.0,5,1.0,post,NaN,NaN,NaN,NaN,...,"Большая аллея Санкт-Петербург Россия,Большая ...","['Большая аллея Санкт-Петербург Россия', 'Бол...",Большая аллея Санкт-Петербург Россия,Большая аллея,"Большая аллея, Острова, округ Чкаловское, Санк...",POINT (30.28604 59.97685),NaN,street,Жалобы на незаконную торговлю,1.0
1,58907,В Центральном районе продолжается борьба с нез...,892.0,5,1.0,post,NaN,NaN,NaN,NaN,...,"Большая аллея Санкт-Петербург Россия,Большая ...","['Большая аллея Санкт-Петербург Россия', 'Бол...",Большая аллея Санкт-Петербург Россия,Большая улица,"Большая аллея, Острова, округ Чкаловское, Санк...",POINT (30.28604 59.97685),NaN,street,Жалобы на незаконную торговлю,1.0
2,58907,В Центральном районе продолжается борьба с нез...,892.0,5,1.0,post,NaN,NaN,NaN,NaN,...,"Большая аллея Санкт-Петербург Россия,Большая ...","['Большая аллея Санкт-Петербург Россия', 'Бол...",Большая аллея Санкт-Петербург Россия,Большой проспект,"Большая аллея, Острова, округ Чкаловское, Санк...",POINT (30.28604 59.97685),NaN,street,Жалобы на незаконную торговлю,1.0
3,58907,В Центральном районе продолжается борьба с нез...,892.0,5,1.0,post,NaN,NaN,NaN,NaN,...,"Большая аллея Санкт-Петербург Россия,Большая ...","['Большая аллея Санкт-Петербург Россия', 'Бол...",Большая улица Санкт-Петербург Россия,Большая аллея,"Большая улица, Тярлево, Санкт-Петербург, Север...",POINT (30.44338 59.70189),NaN,street,Жалобы на незаконную торговлю,1.0
4,58907,В Центральном районе продолжается борьба с нез...,892.0,5,1.0,post,NaN,NaN,NaN,NaN,...,"Большая аллея Санкт-Петербург Россия,Большая ...","['Большая аллея Санкт-Петербург Россия', 'Бол...",Большая улица Санкт-Петербург Россия,Большая улица,"Большая улица, Тярлево, Санкт-Петербург, Север...",POINT (30.44338 59.70189),NaN,street,Жалобы на незаконную торговлю,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,57489,Очень нужны места для купания и прогулок у вод...,NaN,0,NaN,comment,NaN,16164672.0,57475.0,None,...,NaN,NaN,NaN,NaN,NaN,POINT (30.37323 59.94547),Таврический сад,park,unknown_Экология,1.0
383,57410,"[id973044|Екатерина], в прошлом году было боль...",NaN,0,NaN,reply,NaN,2585913.0,57322.0,57371,...,NaN,NaN,NaN,NaN,NaN,POINT (30.37323 59.94547),Таврический сад,park,Жалобы на несвоевременный вывоз отходов,0.84
384,57346,А где находится Некрасовский сад?,NaN,0,NaN,comment,NaN,7038519.0,57317.0,None,...,NaN,NaN,NaN,NaN,NaN,POINT (30.36747 59.93735),Некрасовский сад,park,Зачисление в д/сад,0.709
385,57383,"[id7038519|Светлана Белованова], здравствуйте!...",NaN,0,NaN,reply,NaN,-143265175.0,57317.0,57346,...,NaN,NaN,NaN,NaN,NaN,POINT (30.36747 59.93735),Некрасовский сад,park,unknown_Благоустройство,0.914


In [29]:
percentages = gdf.copy()['cats'].value_counts(normalize=True)[:5] * 100

In [30]:
percentages_df = pd.DataFrame({'Value': percentages.index, 'Percentage': percentages.values})

In [31]:
percentages_df

,Value,Percentage
0,Не ЦУР,25.322997
1,Жалобы на незаконную торговлю,15.245478
2,unknown_Образование,12.919897
3,Вопросы граждан о проектах/планах/сроках/ходе ...,7.751938
4,unknown_Благоустройство,7.235142


In [10]:
df['toponym'] = 'улица'

In [11]:
user_partition = df['from_id'].loc[df['cats'] == 'Не ЦУР'].value_counts(normalize=True) * 100

In [12]:
user_partition = pd.DataFrame({'user_id': user_partition.index, 'Percentage': user_partition.values})

In [13]:
user_partition

,user_id,Percentage
0,1492921.0,20.0
1,967593.0,10.0
2,636.0,10.0
3,650393846.0,10.0
4,871865.0,10.0
5,185082.0,10.0
6,439008.0,10.0
7,632043.0,10.0
8,144340.0,10.0


In [32]:
gdf = City_services().run(gdf, 'text')

In [73]:
gdf_final = EmotionRecognizer().add_emotion_column(gdf, 'text')

c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [74]:
gdf_final

,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,parents_stack,...,only_full_street_name,Location,geometry,other_geo_obj,geo_obj_tag,cats,probs,City_services_extraced,City_services,emotion
0,58907,В Центральном районе продолжается борьба с нез...,892.0,5,1.0,post,NaN,NaN,NaN,NaN,...,Большая аллея,"Большая аллея, Острова, округ Чкаловское, Санк...",POINT (30.28604 59.97685),NaN,street,Жалобы на незаконную торговлю,1.0,[],[],neutral
1,58907,В Центральном районе продолжается борьба с нез...,892.0,5,1.0,post,NaN,NaN,NaN,NaN,...,Большая улица,"Большая аллея, Острова, округ Чкаловское, Санк...",POINT (30.28604 59.97685),NaN,street,Жалобы на незаконную торговлю,1.0,[],[],neutral
2,58907,В Центральном районе продолжается борьба с нез...,892.0,5,1.0,post,NaN,NaN,NaN,NaN,...,Большой проспект,"Большая аллея, Острова, округ Чкаловское, Санк...",POINT (30.28604 59.97685),NaN,street,Жалобы на незаконную торговлю,1.0,[],[],neutral
3,58907,В Центральном районе продолжается борьба с нез...,892.0,5,1.0,post,NaN,NaN,NaN,NaN,...,Большая аллея,"Большая улица, Тярлево, Санкт-Петербург, Север...",POINT (30.44338 59.70189),NaN,street,Жалобы на незаконную торговлю,1.0,[],[],neutral
4,58907,В Центральном районе продолжается борьба с нез...,892.0,5,1.0,post,NaN,NaN,NaN,NaN,...,Большая улица,"Большая улица, Тярлево, Санкт-Петербург, Север...",POINT (30.44338 59.70189),NaN,street,Жалобы на незаконную торговлю,1.0,[],[],neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,57489,Очень нужны места для купания и прогулок у вод...,NaN,0,NaN,comment,NaN,16164672.0,57475.0,None,...,NaN,NaN,POINT (30.37323 59.94547),Таврический сад,park,unknown_Экология,1.0,[садах],[Сад],neutral
383,57410,"[id973044|Екатерина], в прошлом году было боль...",NaN,0,NaN,reply,NaN,2585913.0,57322.0,57371,...,NaN,NaN,POINT (30.37323 59.94547),Таврический сад,park,Жалобы на несвоевременный вывоз отходов,0.84,[саду],[Сад],neutral
384,57346,А где находится Некрасовский сад?,NaN,0,NaN,comment,NaN,7038519.0,57317.0,None,...,NaN,NaN,POINT (30.36747 59.93735),Некрасовский сад,park,Зачисление в д/сад,0.709,[сад],[Сад],neutral
385,57383,"[id7038519|Светлана Белованова], здравствуйте!...",NaN,0,NaN,reply,NaN,-143265175.0,57317.0,57346,...,NaN,NaN,POINT (30.36747 59.93735),Некрасовский сад,park,unknown_Благоустройство,0.914,"[сад, сад, памятник, сада]","[Сад, Сад, Памятник, Сад]",happiness


In [35]:
city_services = []
for x in gdf['City_services'].to_list():
    city_services.extend(x if isinstance(x, list) else [x])
    
city_services = set(city_services)

In [36]:
city_services

{'Аэродром',
 'Бар',
 'Библиотека',
 'Ботанический сад',
 'Газон',
 'Детская площадка',
 'Детский сад',
 'Залив',
 'Заповедник',
 'Зоопарк',
 'Кафе',
 'Лес',
 'Магазин',
 'Метро',
 'Музей',
 'Озеро',
 'Памятник',
 'Парк',
 'Парковка',
 'Поликлиника',
 'Продукты',
 'Река',
 'Роддом',
 'Сад',
 'Спортивная площадка',
 'Театр',
 'Торговый центр',
 'Университет',
 'Фонтан',
 'Цирк',
 'Школа'}

In [38]:
gdf['City_services'] = gdf['City_services'].apply(lambda x: ','.join(set(x)) if x else None)

In [39]:
gdf['City_services']

0              None
1              None
2              None
3              None
4              None
           ...     
381             Сад
382             Сад
383             Сад
384    Памятник,Сад
385    Памятник,Сад
Name: City_services, Length: 386, dtype: object

In [91]:
import numpy as np
df['toponym'] = np.random.randint(0, 3, df.shape[0])

In [92]:
df

,date,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,parents_stack,group_name,cats,probs,toponym,City_services_extraced,City_services
0,2024-05-22 16:47:48,18024,"Дорого и плохо, а порой еще и опасно Сюжет «7...",1104.0,45,7.0,post,https://vk.com/wall-33468323_41130|«собачий,NaN,NaN,NaN,Шумные Шушары,Не ЦУР,0.849,1,"[Корт, Парк, аквапарк, Корт, Корт, Корт]","Аквапарк,Морг,Парк"
1,2024-05-22 13:26:45,18023,26 МАЯ | Открытый детский турнир по хоккею Ув...,374.0,3,1.0,post,https://vk.com/mamo_shushary|муниципалитет],NaN,NaN,NaN,Шумные Шушары,unknown_Благоустройство,0.406,0,[],None
2,2024-05-17 16:51:13,18011,Прямой эфир с [id1745844|Екатериной Краевой] с...,2302.0,26,8.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,unknown_Благоустройство,1.0,2,[кладбища],Кладбище
3,2024-05-16 17:46:54,18003,Показательное отношение Милонова к своим избир...,2515.0,30,11.0,post,https://vk.com/wall-516730_675000|«помощь»],NaN,NaN,NaN,Шумные Шушары,Жалобы на незаконную торговлю,0.991,2,[],None
4,2024-05-14 15:46:33,17992,Шушары Today,3022.0,23,1.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,Жалобы на незаконную торговлю,0.896,1,[],None
5,2024-05-13 01:05:41,17977,"ВНИМАНИЕ!Сегодня, как обычно, без должного инф...",11194.0,26,34.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,unknown_Транспорт,0.956,2,[кладбища],Кладбище
6,2024-05-06 14:10:00,17964,В этом году за счёт муниципального бюджета Шуш...,3149.0,16,17.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,"Отсутствие информирования об изменении, закрыт...",0.989,1,[],None
7,2024-04-27 10:27:31,17937,Сегодня свой День рождения отмечает неотъемлем...,7119.0,213,26.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,Не ЦУР,0.519,0,[],None
8,2024-05-22 13:55:20,18025,А что за движуха сегодня утром была на Новгоро...,NaN,3,NaN,comment,NaN,1502930.0,18024.0,NaN,Шумные Шушары,Критика транспортной реформы,0.688,1,[],None
9,2024-05-22 15:16:35,18026,"[id1502930|Анна], таки да, к нам точно едет ""р...",NaN,7,NaN,reply,NaN,967593.0,18024.0,18025.0,Шумные Шушары,Не ЦУР,0.955,2,[],None


In [96]:
router = {0: 'Первый', 1: 'Второй', 2: None}

In [97]:
df['toponym'] = df['toponym'].apply(lambda x: router[x])

In [34]:
top_toponyms = gdf['only_full_street_name'].value_counts(normalize=True) * 100

In [36]:
top_toponyms[:5]

Невский проспект        16.083916
Невский путепровод      11.538462
Невский переулок        11.538462
улица Чайковского        3.846154
Дворцовая набережная     2.097902
Name: only_full_street_name, dtype: float64

In [37]:
top_toponyms_names = top_toponyms.index

In [52]:
top_toponyms_names[:5]

Index(['Невский проспект', 'Невский путепровод', 'Невский переулок',
       'улица Чайковского', 'Дворцовая набережная'],
      dtype='object')

In [38]:
top_toponyms_df = pd.DataFrame([top_toponyms_names, top_toponyms.to_list()]).T

In [43]:
top_toponyms_df.columns[1]

1

In [49]:
top_toponyms_df.rename(columns={0: 'toponym',
                                1: 'percent'},
                       inplace=True)

In [50]:
top_toponyms_df

,toponym,percent
0,Невский проспект,16.083916
1,Невский путепровод,11.538462
2,Невский переулок,11.538462
3,улица Чайковского,3.846154
4,Дворцовая набережная,2.097902
...,...,...
67,Захарьевская улица,0.34965
68,Садовая улица,0.34965
69,Советская улица,0.34965
70,Народный переулок,0.34965


In [36]:
toponyms = df['toponym'].unique()

In [161]:
result = {}

for i in top_toponyms_names[:5]:
    
    posts_ids = gdf_final['id'].loc[gdf_final['only_full_street_name'] == i].to_list()
    comments_ids = gdf_final['id'].loc[gdf_final['post_id'].isin(posts_ids) & gdf_final['only_full_street_name'].isin([i, None])].to_list()
    replies_ids = gdf_final['id'].loc[gdf_final['parents_stack'].isin(comments_ids) & gdf_final['only_full_street_name'].isin([i, None])].to_list()
    
    all_ids = tuple(sorted(list(set(posts_ids + comments_ids + replies_ids))))
    
    toponym_gdf_final = gdf_final.loc[gdf_final['id'].isin(all_ids)]
    part_users = len(toponym_gdf_final['from_id'].unique())/len(df['from_id'].unique())
    part_messages = len(toponym_gdf_final['id'].unique())/len(df['id'].unique())
    
    info = {'part_users': part_users,
            'part_messages': part_messages}
    
    services = [obj for inner_list in toponym_gdf_final['City_services'].to_list() for obj in inner_list]
    
    if services:
        unique_services = set(services)
        values = [0 for j in unique_services]
        
        services_dict = dict.fromkeys(unique_services, 0)
        
        service_info = {}
        for j in services:
            if j in services_dict:
                services_dict[j] = services_dict[j] + 1
    
    service_gdf = toponym_gdf_final.dropna(subset='City_services')   
         
    for j in unique_services:
        
        service_info = {'counts': 0}
        
        if j in services_dict:
                service_info['counts'] = service_info['counts'] + 1
        
        emotions = service_gdf['emotion'].to_list()
        
        if emotions:
            unique_emotions = set(emotions)
            values = [0 for k in unique_emotions]

            emotions_dict = {key: value for key, value in zip(unique_emotions, values)}
            
            for k in emotions:
                if k in emotions_dict:
                    emotions_dict[k] = emotions_dict[k] + 1
            
        service_info['emotions'] = emotions_dict
    
        services_dict[j] = service_info
        
        info['services'] = services_dict
        
        result[i] = info

c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\pandas\core\algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])
c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\pandas\core\algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])
c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-3_lHyauD-py3.11\Lib\site-packages\pandas\core\algorithms.py:522: DeprecationWarning: np.find_co

In [162]:
result

{'Невский проспект': {'part_users': 0.03821656050955414,
  'part_messages': 0.022263450834879406,
  'services': {'Роддом': {'counts': 1,
    'emotions': {'anger': 9, 'happiness': 10, 'sadness': 11, 'neutral': 82}},
   'Аэродром': {'counts': 1,
    'emotions': {'anger': 9, 'happiness': 10, 'sadness': 11, 'neutral': 82}},
   'Лес': {'counts': 1,
    'emotions': {'anger': 9, 'happiness': 10, 'sadness': 11, 'neutral': 82}},
   'Газон': {'counts': 1,
    'emotions': {'anger': 9, 'happiness': 10, 'sadness': 11, 'neutral': 82}}}},
 'Невский путепровод': {'part_users': 0.022292993630573247,
  'part_messages': 0.01020408163265306,
  'services': {'Аэродром': {'counts': 1,
    'emotions': {'anger': 9, 'happiness': 9, 'sadness': 9, 'neutral': 72}},
   'Газон': {'counts': 1,
    'emotions': {'anger': 9, 'happiness': 9, 'sadness': 9, 'neutral': 72}}}},
 'Невский переулок': {'part_users': 0.022292993630573247,
  'part_messages': 0.01020408163265306,
  'services': {'Аэродром': {'counts': 1,
    'emoti

In [163]:
result_df = pd.DataFrame.from_dict(result)

In [164]:
result_df

,Невский проспект,Невский путепровод,Невский переулок,улица Чайковского,Дворцовая набережная
part_users,0.038217,0.022293,0.022293,0.006369,0.006369
part_messages,0.022263,0.010204,0.010204,0.010204,0.001855
services,"{'Роддом': {'counts': 1, 'emotions': {'anger':...","{'Аэродром': {'counts': 1, 'emotions': {'anger...","{'Аэродром': {'counts': 1, 'emotions': {'anger...","{'Газон': {'counts': 1, 'emotions': {'anger': ...","{'Газон': {'counts': 1, 'emotions': {'anger': ..."


In [129]:
validate = gdf_final.loc[gdf_final['only_full_street_name'] == 'Невский проспект']

In [ ]:
validate

In [ ]:
unique_emotions

{'neutral'}

In [123]:
services_dict

{'Магазин': 1}

In [102]:
emotions_dict

{'Аэродром': 0, 'Газон': 0}

In [66]:
part_messages

0.022263450834879406

In [69]:
services

15             []
16             []
27             []
30             []
33             []
38             []
51             []
54             []
57             []
107      [Роддом]
124            []
127            []
130            []
140       [Газон]
141       [Газон]
146       [Газон]
149       [Газон]
152       [Газон]
155       [Газон]
163            []
166            []
169            []
176            []
179            []
182            []
185    [Аэродром]
188    [Аэродром]
191    [Аэродром]
194            []
197            []
200            []
212            []
215            []
218            []
229            []
236            []
239            []
242            []
246            []
253            []
265         [Лес]
269            []
272            []
275            []
276            []
277            []
Name: City_services, dtype: object

In [67]:
res = []
for i in services:
    res += i
    
res = set(res)

In [68]:
res

{'Аэродром', 'Газон', 'Лес', 'Роддом'}

In [161]:
count_dict = {}

for i in res:
    count_dict[i] = 0

In [171]:
df['City_services_extraced'].iloc[0]

['Корт', 'Парк', 'аквапарк', 'Корт', 'Корт', 'Корт']

In [180]:
for i in range(len(df)):
    for j in set(df['City_services_extraced'].iloc[i]):
        if j in count_dict:
            count_dict[j] = count_dict[j] + 1

In [184]:
service_df_list = [[key, value] for key, value in count_dict.items()]

In [189]:
services_df = pd.DataFrame(service_df_list, columns=['service', 'count'])

In [191]:
services_df.sort_values(by='count', ascending=False)

,service,count
4,кладбища,5
0,горы,1
1,Парк,1
2,Корт,1
3,аквапарк,1


In [119]:
toponym_df = df.loc[df['id'].isin(all_ids)]
part_users = len(toponym_df['from_id'].unique())/len(df['from_id'].unique())
part_messages = len(toponym_df['id'].unique())/len(df['id'].unique())

In [120]:
part_users

0.47058823529411764

In [121]:
part_messages

0.3220338983050847